In [2]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,make_scorer
# from sklearn.linear_model import Lasso 
# from sklearn.linear_model import Ridge 
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# from google.colab import  drive 
# drive.mount('/drive')

C:\Users\Zach\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [7]:
# box_scores_and_features = pd.read_csv("/drive/My Drive/Capstone Project College Basketball Data/ESPN_data/Features_and_boxscores/player_and_opp_stats_with_position.csv",index_col = 0)

# pd.set_option('max_columns', None)
# box_scores_and_features = box_scores_and_features.drop_duplicates()
# box_scores_and_features.head()
# ##### Sample_teams_for_github/streamlit
# sample_teams = ["Michigan","Duke","North Carolina","Kansas","Gonzaga","Purdue","Kentucky","Auburn","Tennessee","Baylor",'Texas Tech',"Villanova","Wisconsin"]
# sample_df_box_score_and_features = box_scores_and_features[box_scores_and_features["team_short_display_name"].isin(sample_teams)]
# sample_df_box_score_and_features.to_csv("src/sample_player_and_opp_stats_with_position.csv")
sample_df_box_score_and_features= pd.read_csv("src/sample_player_and_opp_stats_with_position.csv")

In [8]:
pd.options.mode.chained_assignment = None
### Using 2022 as test year. Train on a sample of the other years  
training_df =  sample_df_box_score_and_features[sample_df_box_score_and_features["season"]<2022].sample(5000,random_state=698)
testing_df = sample_df_box_score_and_features[sample_df_box_score_and_features["season"]==2022]
### fill in colors for missing teams - Dark Grey 
testing_df["team_color"] = testing_df.loc[:,"team_color"].fillna(value='#555555')


# pd.options.mode.chained_assignment = None
# ### Using 2022 as test year. Train on a sample of the other years  
# training_df =  box_scores_and_features[box_scores_and_features["season"]<2022].sample(5000,random_state=698)
# testing_df = box_scores_and_features[box_scores_and_features["season"]==2022]
# ### fill in colors for missing teams - Dark Grey 
# testing_df["team_color"] = testing_df.loc[:,"team_color"].fillna(value='#555555')

In [9]:
points_model_df = testing_df[testing_df["team_short_display_name"]== "Michigan"]
points_model_df

,Unnamed: 0,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,blk,to,pf,pts,starter,ejected,did_not_play,active,athlete_jersey,athlete_id,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_name,team_logo,team_id,team_abbreviation,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev,points_per_min,prev_player_ppg,prev_player_points_per_min,game_total_points,game_number,cum_average_opp_points_to_role,cum_average_opp_min_allowed_to_role,prev_3_games_opp_min_allowed_to_role,prev_3_games__opp_points_allowed_to_role,total_pts_allowed,prev_total_opp_ppg,Home,Starter_numeric,Cleaned_position_bigs_guards,prev_avg_pts_to_role_pos,prev_avg_min_to_role_position,ATH,Big_man,Guard
1796,50478,Michigan,401364348,"{""DeVante' Jones"", 'Caleb Houstan', 'Hunter Di...",Caleb Houstan,27,0,6,6,3,2,0,3,2,16,True,False,False,False,22.0,4433623,C. Houstan,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2021-12-08,6,9,4,7,0,0,35.0,HOME,158,NEB,0.592593,NaN,NaN,102,1,51.889,29.044,32.600,60.000,102,72.333,1,1,Big_man,22.471,27.046,0,1,0
1797,50479,Michigan,401364348,"{""DeVante' Jones"", 'Caleb Houstan', 'Hunter Di...",Brandon Johns Jr.,21,1,4,5,0,1,0,2,3,20,True,False,False,False,23.0,4397206,B. Johns Jr.,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2021-12-08,6,9,2,3,6,7,35.0,HOME,158,NEB,0.952381,NaN,NaN,102,1,51.889,29.044,32.600,60.000,102,72.333,1,1,Big_man,22.471,27.046,0,1,0
1798,50480,Michigan,401364348,"{""DeVante' Jones"", 'Caleb Houstan', 'Hunter Di...",Hunter Dickinson,29,6,6,12,3,0,1,2,2,15,True,False,False,False,1.0,4432180,H. Dickinson,https://a.espncdn.com/i/headshots/mens-college...,Center,C,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2021-12-08,7,14,1,3,0,0,35.0,HOME,158,NEB,0.517241,NaN,NaN,102,1,51.889,29.044,32.600,60.000,102,72.333,1,1,Big_man,22.471,27.046,0,1,0
1799,50481,Michigan,401364348,"{""DeVante' Jones"", 'Caleb Houstan', 'Hunter Di...",DeVante' Jones,25,2,3,5,8,3,1,0,2,3,True,False,False,False,12.0,4397506,D. Jones,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2021-12-08,1,5,1,3,0,0,35.0,HOME,158,NEB,0.120000,NaN,NaN,102,1,51.889,29.044,32.600,60.000,102,72.333,1,1,Guard,32.965,29.301,0,0,1
1800,50482,Michigan,401364348,"{""DeVante' Jones"", 'Caleb Houstan', 'Hunter Di...",Eli Brooks,28,1,2,3,2,0,0,0,1,10,True,False,False,False,55.0,4277960,E. Brooks,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2021-12-08,3,10,2,8,2,2,35.0,HOME,158,NEB,0.357143,NaN,NaN,102,1,51.889,29.044,32.600,60.000,102,72.333,1,1,Guard,32.965,29.301,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,50755,Michigan,401412488,"{""DeVante' Jones"", 'Moussa Diabate', 'Caleb Ho...",Jaron Faulds,2,0,0,0,0,1,0,0,2,1,False,False,False,False,44.0,4279471,J. Faulds,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2022-03-02,0,0,0,0,1,2,21.0,AWAY,127,MSU,0.500000,0.92,0.298,87,28,15.556,11.960,11.544,17.333,87,67.250,0,0,Big_man,6.705,11.457,0,1,0
2074,50756,Michigan,401412488,"{""DeVante' Jones"", 'Moussa Diabate', 'Caleb Ho...",Kobe Bufkin,9,0,0,0,2,1,0,0,1,0,False,False,False,False,2.0,4683736,K. Bufkin,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Wolverines,https://a.espnc

Data Preprocessing

* Columns with "prev_" are the cumulative avgs shifted to obtain the averages before each game.
* Role = Starter/reserve
* Position = Converted all "Center" positions to "bigs" to have all 'forwards' and 'centers' the same, while having point guards and shooting guards as guards



In [14]:
#### Columns to use for model 
def transform_training_testing(dataframe):
    columns = ["prev_player_ppg","prev_player_points_per_min",	"game_number",
    "cum_average_opp_points_to_role","cum_average_opp_min_allowed_to_role",
    "prev_3_games_opp_min_allowed_to_role",
    "prev_3_games__opp_points_allowed_to_role",
    "prev_total_opp_ppg","Home","Starter_numeric",	
    "prev_avg_pts_to_role_pos","prev_avg_min_to_role_position",
    "Big_man","Guard",
    "pts"]

    points_df_model =dataframe.loc[:,columns] 
    points_df_model = points_df_model.rename(columns = {"prev_player_ppg":"prev_avg_player_ppg","prev_player_points_per_min":"prev_avg_player_points_per_min"})
    points_df_model = points_df_model.dropna()
    return points_df_model

training_df_processed = transform_training_testing(training_df)
testing_df_processed = transform_training_testing(testing_df)

In [15]:
def convert_to_training_testing(training_df_processed,testing_df_processed):
    X_train= training_df_processed.loc[:, ~training_df_processed.columns.isin(['pts'])]               
    y_train = training_df_processed["pts"]
    X_test= testing_df_processed.loc[:, ~testing_df_processed.columns.isin(['pts'])]               
    y_test = testing_df_processed["pts"]

    # X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
    X_train_scaled = MinMaxScaler().fit_transform(X_train)
    X_test_scaled= MinMaxScaler().fit_transform(X_test)
    return X_train,y_train,X_test,y_test,X_train_scaled,X_test_scaled
X_train,y_train,X_test,y_test,X_train_scaled,X_test_scaled = convert_to_training_testing(training_df_processed,testing_df_processed)


#### Uncomment to test for overfitting
# kfold = KFold(n_splits=5)
# X = points_df_model.loc[:, ~points_df_model.columns.isin(['pts'])]               
# y = points_df_model["pts"]
# X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
# X_train_scaled = MinMaxScaler().fit_transform(X_train)
# X_test_scaled= MinMaxScaler().fit_transform(X_test)
# X_train

In [17]:
#### Linear Regression has the lowest MAE compared to the other models tested
linear_reg = LinearRegression().fit(X_train_scaled,y_train)
y_pred = linear_reg.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)

model_name = "Linear reg"
print(model_name)
print("Mean Absolute Error: {}".format(mae))
pts_predictions_df = y_test.to_frame().rename(columns={"pts":"pts_label"})
pts_predictions_df['points_prediction'] = y_pred
merged_predictions = pd.merge(pts_predictions_df,sample_df_box_score_and_features,left_index= True, right_index= True)

merged_predictions.sample(20)
### Uncomment for sample team
# test_boxscore[test_boxscore.team_short_display_name=='Michigan']

Linear reg
Mean Absolute Error: 3.7871385842472582


,pts_label,points_prediction,Unnamed: 0,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,blk,to,pf,pts,starter,ejected,did_not_play,active,athlete_jersey,athlete_id,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_name,team_logo,team_id,team_abbreviation,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev,points_per_min,prev_player_ppg,prev_player_points_per_min,game_total_points,game_number,cum_average_opp_points_to_role,cum_average_opp_min_allowed_to_role,prev_3_games_opp_min_allowed_to_role,prev_3_games__opp_points_allowed_to_role,total_pts_allowed,prev_total_opp_ppg,Home,Starter_numeric,Cleaned_position_bigs_guards,prev_avg_pts_to_role_pos,prev_avg_min_to_role_position,ATH,Big_man,Guard
318,13,11.375,5038,Auburn,401389430,"{'Walker Kessler', 'Zep Jasper', 'Devan Cambri...",Walker Kessler,25,5,5,10,0,2,4,0,2,13,True,False,False,False,13.0,4433136,W. Kessler,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2022,2,2021-11-25,5,12,1,3,2,5,10.0,HOME,2350,LUC,0.520000,11.79,0.432,62,29,44.200,26.480,27.733,45.333,62,60.800,1,1,Big_man,13.857,26.030,0,1,0
1306,19,11.250,39624,Kansas,401368420,"{'Dajuan Harris Jr.', 'David McCormack', 'Jale...",David McCormack,25,5,6,11,2,0,0,3,4,19,True,False,False,False,33.0,4397019,D. McCormack,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Jayhawks,https://a.espncdn.com/i/teamlogos/ncaa/500/230...,2305,KU,0022B4,2022,2,2022-02-20,7,12,0,0,5,7,13.0,HOME,277,WVU,0.760000,10.46,0.433,71,14,50.458,27.550,25.067,51.667,71,66.583,1,1,Big_man,16.453,23.802,0,1,0
3469,0,1.375,98421,Villanova,401373550,"{'Jermaine Samuels', 'Collin Gillespie', 'Eric...",Jordan Longino,17,1,0,1,1,1,0,1,3,0,False,False,False,False,15.0,4684153,J. Longino,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Wildcats,https://a.espncdn.com/i/teamlogos/ncaa/500/222...,222,VILL,123d7C,2022,2,2022-01-16,0,4,0,0,0,0,40.0,AWAY,2086,BUT,0.000000,1.18,0.133,82,16,20.846,12.432,14.667,22.333,82,63.615,0,0,Guard,8.596,13.302,0,0,1
732,15,16.500,22371,Duke,401369863,"{'AJ Griffin', 'Mark Williams', 'Wendell Moore...",Wendell Moore Jr.,33,1,3,4,1,1,0,4,2,15,True,False,False,False,0.0,4592187,W. Moore Jr.,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Blue Devils,https://a.espncdn.com/i/teamlogos/ncaa/500/150...,150,DUKE,001A57,2022,2,2022-01-15,6,8,0,0,3,3,19.0,AWAY,152,NCST,0.454545,16.31,0.507,88,14,50.000,27.659,27.000,40.667,88,74.176,0,1,Big_man,19.489,25.847,0,1,0
726,12,11.000,22365,Duke,401369862,"{'Jeremy Roach', 'Mark Williams', 'Wendell Moo...",Mark Williams,27,2,5,7,2,0,6,3,3,12,True,False,False,False,15.0,4701232,M. Williams,https://a.espncdn.com/i/headshots/mens-college...,Center,C,Blue Devils,https://a.espncdn.com/i/teamlogos/ncaa/500/150...,150,DUKE,001A57,2022,2,2022-01-09,6,8,0,0,0,1,8.0,AWAY,2390,MIA,0.444444,8.58,0.392,74,13,54.933,28.933,31.600,70.000,74,73.733,0,1,Big_man,20.494,25.570,0,1,0
1643,0,1.875,41078,Kentucky,401364953,"{'Jacob Toppin', 'Kellan Grady', 'Oscar Tshieb...",Daimion Collins,0,0,0,0,0,0,0,0,0,0,False,False,False,False,4.0,4611412,D. Collins,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wildcats,https://a.espncdn.com/i/teamlogos/ncaa/500/96.png,96,UK,005DAA,2022,2,2022-02-26,0,0,0,0,0,0,4.0,HOME,8,ARK,NaN,1.67,0.306,73,17,20.536,13.734,15.111,14.333,73,67.357,1,0,Big_man,7.981,12.501,0,1,0
2871,16,15.500,85489,Tennessee,401364959,"{'Brandon Huntley-Hatfield', 'Santiago Vescovi...",Kennedy Chandler,35,0,1,1,2,4,0,4,0,16,True,False,False,False,1.0,4432646,K. Chandler,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Volunteers,https://a.espncdn.com/i/teamlogos/ncaa/500/263...,2633,TENN,EE9627,2022,2,2022-03-01,6,13,2,5,2,3,15.0,HOME,61,UGA,0.457143,13.25,0.417,75,18,56.793,27.229,28.667,70.000,75,78.345,1,1,Gu

Prediction Visualization

In [20]:
def display_prediction_graph(player_pred):#line= True, bars=True,head=True,logo= True):
    # player_pred = merged_predictions[merged_predictions.athlete_display_name == player]

    line = alt.Chart(player_pred).mark_line(strokeDash=[1,1],color= 'black').encode(
      x='game_date:T',
      y=alt.Y('points_prediction'),
      tooltip = ["game_date:T",'points_prediction','pts',"min","starter"]
      )
    bars = alt.Chart(player_pred).mark_bar().encode(
      x=alt.X('game_date:T', title = 'Game Date'),
      y=alt.Y('pts', title = "Points"),
      color = alt.Color('team_color', legend = None),
      tooltip = ["game_date:T",'points_prediction','pts',"min","starter"]
      )
    try:
        logo = alt.Chart(player_pred).mark_image().encode(
          x=alt.value(500), x2=alt.value(600),  # pixels from left
          y=alt.value(0), y2=alt.value(75),  # pixels from top
          url='team_logo'
        )
    except:
        return (line+bars)

    try:
        head_shot_image = alt.Chart(player_pred).mark_image().encode(
        x=alt.value(500), x2=alt.value(600),  # from left
        y=alt.value(400), y2=alt.value(500),  # from top
        url='athlete_headshot_href'
        ).properties(
            width = 500,
            height  =500,
            title = "Points (Bars) and Predicted Points (Dashed line)"
        )
        return (line+bars+logo+head_shot_image)
    except:
        return (line+bars+logo)  

In [21]:
# team = 'Michigan'

### Uncomment other players
# player = 'Hunter Dickinson'
# player = 'Paolo Banchero'
player = "Ochai Agbaji"
### Filter on team, player, 
# team_predictions = merged_predictions[merged_predictions.team_short_display_name==team] 

#### If statements when the images are null. 
player_pred = merged_predictions[merged_predictions.athlete_display_name == player]
display_prediction_graph(player_pred)

alt.LayerChart(...)

Other models considered

In [ ]:
# #### Uncomment below for overfitting
# # name_model = "Lasso"
# # scorer_name = "mean abs error"
# # scorer_func= mean_absolute_error
# # print(name_model)
# # alphas= np.arange(0.01,.1,.02)
# # param_grid = dict(alpha =alphas)
# # scorer = make_scorer(scorer_func)
# # grid = GridSearchCV(Lasso(),param_grid=param_grid, scoring = scorer,cv = 5)
# # grid_result = grid.fit(X_train_scaled, y_train)
# # # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# # print("best param: {}".format(grid_result.best_params_))

# # best_alpha = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(Lasso(alpha=best_alpha),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(str(scorer_name),result.mean()))
# # # grid_result.get_params
# Lasso_model = (Lasso(alpha =  0.0899).fit(X_train_scaled,y_train))
# y_pred = Lasso_model.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)

# model_name = "Lasso"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))



# # merged_predictions[merged_predictions.team_short_display_name=='Michigan']

# ### uncomment for k-fold and train_test_split
# # pts_predictions_df = y_test.to_frame().rename(columns={"pts":"pts_label"})
# # pts_predictions_df['points_prediction'] = y_pred
# # sampled_player_games = pts_predictions_df.sample(5000,random_state = 150)
# # merged_predictions = pd.merge(sampled_player_games,box_scores_and_features,left_index= True, right_index= True, how = "left")
# # merged_predictions[merged_predictions.team_short_display_name=='Michigan']

In [ ]:
# # name_model = "Ridge"
# # print(name_model)
# # scorer_name = "mean abs error"
# # scorer_func= mean_absolute_error
# # alphas= np.arange(0.1,100,10)
# # param_grid = dict(alpha =alphas)
# # scorer = make_scorer(scorer_func)
# # grid = GridSearchCV(Ridge(),param_grid=param_grid, scoring = scorer,cv = 5)
# # grid_result = grid.fit(X_train_scaled, y_train)
# # # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# # print("best param: {}".format(grid_result.best_params_))

# # best_alpha = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(Ridge(alpha=best_alpha),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(scorer_name,result.mean()))

# Ridge_model = (Ridge(alpha =  90.1).fit(X_train_scaled,y_train))
# y_pred = Ridge_model.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)

# model_name = "Ridge"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))

In [ ]:
# #### Use less data to gridsearch params when using train_test_split on all seasons

# columns = ["prev_player_ppg","prev_player_points_per_min",	"game_number",
# "cum_average_opp_points_to_role","cum_average_opp_min_allowed_to_role",
# "prev_3_games_opp_min_allowed_to_role",
# "prev_3_games__opp_points_allowed_to_role",
# "prev_total_opp_ppg","Home","Starter_numeric",	
# "prev_avg_pts_to_role_pos","prev_avg_min_to_role_position",
# "Big_man","Guard",
# "pts"]
# sample_df = box_scores_and_features[box_scores_and_features["season"]==2019][:5000]
# sample_points_df_model =sample_df.loc[:,columns] 
# #### Previous columns are the cumulative avgs shifted to obtain the averages before each game. ### Column names might need to be renamed 
# #### Role = Starter/reserve
# ### Position - Converted all "Center" positions to "bigs" to have all 'forwards' and 'centers' the same, while having point guards and shooting guards as guards
# sample_points_df_model = sample_points_df_model.rename(columns = {"prev_player_ppg":"prev_avg_player_ppg","prev_player_points_per_min":"prev_avg_player_points_per_min"})

# sample_points_df_model = sample_points_df_model.dropna()
# kfold = KFold(n_splits=5)
# sample_X = sample_points_df_model.loc[:, ~sample_points_df_model.columns.isin(['pts'])]               
# sample_y = sample_points_df_model["pts"]
# sample_X_train, sample_X_test, sample_y_train, sample_y_test = train_test_split(sample_X,sample_y,random_state=0)
# sample_X_train_scaled = MinMaxScaler().fit_transform(sample_X_train)
# sample_X_test_scaled= MinMaxScaler().fit_transform(sample_X_test)
# sample_X_train

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# # name_model = "GradientBoostingRegressor"
# # scorer_name = "mean abs error"
# # scorer_func= mean_absolute_error
# # scorer = make_scorer(scorer_func)
# param_grid = {
#     "loss":["absolute_error"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#     # "min_samples_split": np.arange(1,7,2),
#     # "min_samples_leaf":  np.arange(1,7,2),
#     "max_depth":[8],
#     # "max_features":["log2","sqrt"],
#     "subsample":[.25,0.5,.75, 1.0],
#     }
# grid = GridSearchCV(GradientBoostingRegressor(random_state=0),param_grid=param_grid, scoring = scorer,cv = kfold)
# grid_result = grid.fit(sample_X_train_scaled, sample_y_train)
# # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# print("best param: {}".format(grid_result.best_params_))
# # GradientBoostingModel = GradientBoostingRegressor()
# # GradientBoostingModel = GradientBoostingModel.set_params(**grid.best_params_).fit(X_train_scaled,y_train)
# # best_depth = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(RandomForestRegressor(max_depth=best_depth,random_state=0),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(scorer_name,result.mean()))

# GradientBoostingModel = GradientBoostingRegressor()
# GradientBoostingModel = GradientBoostingModel.set_params(**grid.best_params_).fit(X_train_scaled,y_train)
# y_pred = GradientBoostingModel.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)
# # mse = mean_squared_error(y_test, y_pred)
# # print(depth, r2_train, r2_test,mae)
# model_name = "GradientBoostingModel"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))

In [ ]:
# # name_model = "RandomForestRegressor"
# # scorer_name = "mean abs error"
# # depths = np.arange(1,10,1)
# # param_grid = dict(max_depth =depths)
# # scorer = make_scorer(r2_score)
# # grid = GridSearchCV(RandomForestRegressor(random_state=0),param_grid=param_grid, scoring = scorer,cv = kfold)
# # grid_result = grid.fit(X_train_scaled, y_train)
# # # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# # print("best param: {}".format(grid_result.best_params_))
# # best_depth = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(RandomForestRegressor(max_depth=best_depth,random_state=0),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(scorer_name,result.mean()))

# # for num in np.arange(8,12,1):
# # print(num)
# RFmodel = (RandomForestRegressor(max_depth=9,random_state=0).fit(X_train_scaled,y_train))
# y_pred = RFmodel.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)
# # mse = mean_squared_error(y_test, y_pred)
# # print(depth, r2_train, r2_test,mae)
# model_name = "Random Forest"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))

# pts_predictions_df = y_test.to_frame().rename(columns={"pts":"pts_label"})
# pts_predictions_df['points_prediction'] = y_pred
# merged_predictions = pd.merge(pts_predictions_df,box_scores_and_features,left_index= True, right_index= True,how = 'left')
# merged_predictions[merged_predictions.team_short_display_name=='Michigan']
